#### Simulating Vikbladh et al. (2024) consolidation task

Consolidation can be thought of as training a neocortical generative / predictive model on replayed hippocampal memories. For sequential data, the generative model could correspond to an autoregressive sequence model like GPT-2 that learns to predict the next item in the sequence (by minimising the prediction error on sequences from the training data). The stimuli for Oliver's task can be represented as sequences of form 'START: yellow fruit, STOP: red, REWARD: animal, SEQUENCE: yellow fruit (-1), green animal (2), red animal (2)', which makes it straightforward to train GPT-2.

This notebook simulates the task as follows:
* Pre-train model so that it learns the *rules* of task
* Train on stimuli for task, representing consolidation
* Compare generative model accept / reject performance before and after consolidation

#### Installation / imports:

In [ ]:
! pip install git+https://github.com/huggingface/transformers --upgrade
! pip install accelerate evaluate wonderwords --upgrade
! pip install huggingface_hub --upgrade

In [ ]:
import random
import pandas as pd
import networkx as nx
import logging
from random import shuffle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import random
import string
import os
import re
import glob
import torch
from wonderwords import RandomWord
import os
import gc
import pickle
from sklearn.linear_model import LinearRegression
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from scipy.stats import pearsonr
import math
import seaborn as sns
from scipy.stats import pearsonr

os.environ["WANDB_DISABLED"] = "true"

Define a class for loading a model from a directory, and generating outputs given some input:

In [ ]:
class GPT:

    def __init__(self, base_model):
        self.tokenizer = GPT2Tokenizer.from_pretrained(base_model)
        self.model = GPT2LMHeadModel.from_pretrained(base_model)

    def continue_input(self, input_sequence, max_length=200, num_return_sequences=1, no_repeat_ngram_size=0,
                       do_sample=False, temperature=0.7, num_beams=1):
        
        input_ids = self.tokenizer.encode(input_sequence, return_tensors='pt')
        max_length = len(input_ids[0]) + 100        

        # Generate text
        output = self.model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            num_beams=num_beams,
            no_repeat_ngram_size=no_repeat_ngram_size,
            do_sample=do_sample,
            temperature=temperature,
        )

        # Decode the output
        sequence = output[0].tolist()
        text = self.tokenizer.decode(sequence)
        return text

#### Pre-train model on arbitrary stimuli to learn rules of task

The get_random_stimuli() function generates a random set of nouns and adjectives (3 for each by default). The stimuli are all possible combinations, e.g. for the adjectives ABC and nouns DEF, the stimuli are AD, AE, AF, BD, etc. The get_stimuli() function is the equivalent but for Oliver's task stimuli. 

The get_reward() function predicts reward points for a sequence of stimuli. Given a list of stimuli in random order, the stimulus at which the sequence starts, the adjective at which the sequence ends, and the noun that gives 2 points of reward, the function returns a list of stimuli and their rewards, e.g. ['small chair (2)', 'angry chair (2)', 'metal spoon (-1)'].

In [ ]:
def get_random_stimuli(n=3):
    r = RandomWord()
    adjectives = [r.word(include_parts_of_speech=["adjectives"]).replace(" ", "_") for _ in range(n)]
    nouns = [r.word(include_parts_of_speech=["nouns"]).replace(" ", "_") for _ in range(n)]

    stimuli = []
    for i, noun in enumerate(nouns):
        for adjective in adjectives:
            stimuli.append(f"{adjective} {noun}")

    return stimuli, nouns, adjectives

def get_stimuli():
    stimuli = ["red animal", 
               "green animal", 
               "yellow animal", 
               "red vehicle", 
               "green vehicle", 
               "yellow vehicle", 
               "red fruit", 
               "green fruit", 
               "yellow fruit"]
    
    objects = [word.split()[1] for word in stimuli]
    colours = list(set([word.split()[0] for word in stimuli]))
    return stimuli, objects, colours

def shuffle_stimuli(stimuli):
    random.shuffle(stimuli)
    return stimuli

def get_reward(stimuli, start, stop, reward):
    """Predict reward points for a sequence of stimuli.
    
    Args:
        stimuli (list): List of stimuli in random order.
        start (str): Object at which the sequence starts.
        stop (str): Colour at which the sequence ends.
        reward (str): Category of object that brings 2 points of reward.

    Returns:
        list of str: Reward points descriptions for the sequence.
    """
    points = []
    sequence_started = False
    
    index = 0  # start index
    while True:
        stim = stimuli[index % len(stimuli)]
        colour, obj = stim.split()

        if not sequence_started:
            if stim == start:
                sequence_started = True
            index += 1
            continue
        else:
            if obj == reward:
                points.append(f"{stim} (2)")
            else:
                points.append(f"{stim} (-1)")
    
            if colour == stop:
                break

        index += 1  # move to the next stimulus

    return points

We now generate training and test data for the pre-training.

In [ ]:
training_strs = []
for i in range(1000):
    stimuli, objects, colours = get_random_stimuli()
    stimuli = shuffle_stimuli(stimuli)
    print(f"Stimuli: {stimuli}")
    strs_for_stimuli = []
    for stim in stimuli:
        for colour in colours:
            for obj in objects:
                start = stim
                stop = colour
                reward = obj
                instruction_str = f"START: {start}, STOP: {stop}, REWARD: {reward},"
                points = get_reward(stimuli, start, stop, reward)
                training_str = instruction_str + " SEQUENCE: " + ", ".join(points)
                strs_for_stimuli.append(training_str)
    shuffle(strs_for_stimuli)
    training_strs.extend(strs_for_stimuli)

testing_strs = []
for i in range(10):
    stimuli, objects, colours = get_random_stimuli()
    stimuli = shuffle_stimuli(stimuli)
    print(f"Stimuli: {stimuli}")
    for stim in stimuli:
        for colour in colours:
            for obj in objects:
                start = stim
                stop = colour
                reward = obj
                instruction_str = f"START: {start}, STOP: {stop}, REWARD: {reward},"
                points = get_reward(stimuli, start, stop, reward)
                testing_str = instruction_str + " SEQUENCE: " + ", ".join(points)
                testing_strs.append(testing_str)

print(f"{len(training_strs)} sequences of arbitrary stimuli generated for pre-training.")

The function below runs a script to fine-tune a gpt-2 model on the arbitrary stimuli.

The name_or_path argument is which model to fine-tune from. In the pre-training stage, this will be set to 'gpt2'.

In [ ]:
def train_model_script(name_or_path='rule_model', 
                       num_epochs=3,
                       output_dir='./clm_script',
                       save_steps=100,
                       lr=5e-05, 
                       seed=0):
    torch.cuda.empty_cache()
    gc.collect()
    ! python3 ./run_clm.py \
        --model_name_or_path {name_or_path} \
        --train_file {os.path.join(output_dir, 'train.txt')} \
        --validation_file {os.path.join(output_dir, 'train.txt')} \
        --per_device_train_batch_size 1 \
        --per_device_eval_batch_size 1 \
        --do_train \
        --do_eval \
        --output_dir {output_dir} \
        --overwrite_output_dir \
        --num_train_epochs {num_epochs} \
        --save_strategy 'steps' \
        --save_steps {save_steps} \
        --learning_rate {lr} \
        --seed {seed}
    

Shuffle the data, write it to train.txt and test.txt files, and train gpt2:

In [ ]:
!rm -rf rule_model
!mkdir rule_model

text_file = open("rule_model/train.txt", "w")
n = text_file.write('\n'.join(training_strs))
text_file.close()

text_file = open("rule_model/test.txt", "w")
n = text_file.write('\n'.join(testing_strs))
text_file.close()

train_model_script(name_or_path='gpt2', output_dir='rule_model', num_epochs=10, save_steps=2000)

Test the output with some random start / stop / reward conditions not seen in the training data to see if it's learned the rules:

In [ ]:
model = GPT(base_model='rule_model')

Can the model generalise the rules to new stimuli?

In [ ]:
out = model.continue_input("START: blue table, STOP: green, REWARD: bug, SEQUENCE:", do_sample=False)
print(out)

#### Simulate the task

The simulate_task() function does the following:
* First it clears down the model directory.
* Then it shuffles the stimuli for the trial into a random order.
* Then it generates all possible start / stop / reward combinations and gets the sequence for each from the get_reward() function.
* As above, it turns these into strings with format f"START: ..., STOP: ..., REWARD: ..., SEQUENCE: ..."
* In the reward revaluation case, we train on 2/3 of the reward categories in each trial, and test on a held-out reward category. In the transition revaluation case, we train on 2/3 of the stop colours in each trial, and test on a held-out transition category. 
* The training and test data is saved to files, and a model is trained by calling the train_model_script() 

In [ ]:
def simulate_task(seed=0, num_new=1000, num_epochs=3):
    random.seed(seed)
    training_strs = []
    
    # get stimuli etc and shuffle them
    stimuli, objects, colours = get_stimuli()
    stimuli = shuffle_stimuli(stimuli)
    print(f"Stimuli: {stimuli}")
    
    for stim in stimuli:
        for colour in colours:
            for obj in objects:
                start = stim
                stop = colour
                reward = obj
                instruction_str = f"START: {start}, STOP: {stop}, REWARD: {reward},"
                points = get_reward(stimuli, start, stop, reward)
                training_str = instruction_str + " SEQUENCE: " + ", ".join(points)
                training_strs.append(training_str)

    training_strs = list(set(training_strs))

    random_category = random.choice(objects)
    random_colour = random.choice(colours)
    train_set = [s for s in training_strs if f'REWARD: {random_category}' in s and f'STOP: {random_colour}' in s]
    test_set_reward = [s for s in training_strs if f'REWARD: {random_category}' not in s and f'STOP: {random_colour}' in s]
    test_set_transition = [s for s in training_strs if f'REWARD: {random_category}' in s and f'STOP: {random_colour}' not in s]
    test_set_both = [s for s in training_strs if f'REWARD: {random_category}' not in s and f'STOP: {random_colour}' not in s]
    
    # oversampling trick to avoid overfitting to sequence order
    train_set = np.random.choice(train_set, num_new).tolist()

    output_dir = f'clm_script_{seed}'
    ! rm -rf {output_dir}
    ! mkdir {output_dir}
    
    trial_info = {'stimuli': stimuli, 'train_reward': random_category, 'train_stop': random_colour}
    with open(os.path.join(output_dir, 'trial_info.pkl'), 'wb') as handle:
        pickle.dump(trial_info, handle)

    text_file = open(os.path.join(output_dir, 'train.txt'), "w")
    n = text_file.write('\n'.join(train_set))
    text_file.close()

    text_file = open(os.path.join(output_dir, 'test_reward.txt'), "w")
    n = text_file.write('\n'.join(test_set_reward))
    text_file.close()

    text_file = open(os.path.join(output_dir, 'test_transition.txt'), "w")
    n = text_file.write('\n'.join(test_set_transition))
    text_file.close()

    text_file = open(os.path.join(output_dir, 'test_both.txt'), "w")
    n = text_file.write('\n'.join(test_set_both))
    text_file.close()

    train_model_script(name_or_path='rule_model', 
                       num_epochs=num_epochs, 
                       output_dir=output_dir,
                       save_steps=100,
                       seed=seed)

Reward and transition revaluation test functions:

In [ ]:
def get_accept_reject_choice(seq):
    seq = seq[seq.index('SEQUENCE:') + len('SEQUENCE:'):]
    if 'START' in seq:
        seq = seq[0:seq.index('START')]
    print(seq)
    try:
        numbers = re.findall(r'\([A-Za-z0-9_-]+\)', seq)
        numbers = [int(num.replace('(', '').replace(')', '')) for num in numbers]
        print(numbers)
    except Exception as e:
        "Couldn't convert to int, setting list to []"
        numbers = []
    if sum(numbers) > 0:
        return 1
    elif sum(numbers) < 0:
        return 0
    elif sum(numbers) == 0:
        return 0.5

def test_revaluation(seqs, model):
    result_bools = []
    result_preds = []
    for seq in seqs:
        print("Get true accept / reject:")
        true_a_v_r = get_accept_reject_choice(seq)
        print(true_a_v_r)
        input_str = seq[0:seq.index('SEQUENCE:') + len('SEQUENCE:')]
        continuation = model.continue_input(input_str, do_sample=False)
        print("Get pred accept / reject:")
        pred_a_v_r = get_accept_reject_choice(continuation)
        print(pred_a_v_r)
        result_preds.append(pred_a_v_r)
        if true_a_v_r == pred_a_v_r:
            print("match")
            result_bools.append(1)
        else:
            print("no match")
            result_bools.append(0)
    return (result_bools, result_preds)

def get_len(seq):
    return seq.count(',') - 2

def get_mean(val):
    return np.mean(val[0])

def num_sort(test_string):
    return list(map(int, re.findall(r'\d+', test_string)))[0]

def test_consolidation(base_dir, task_type='transition'):
    print(task_type)
    print("------------------------")
    checkpoints = glob.glob(os.path.join(base_dir, 'checkpoint*'))
    model_dirs = ['rule_model'] + checkpoints 
    print(model_dirs)

    with open(os.path.join(base_dir, f'test_{task_type}.txt')) as f:
        seqs = f.readlines()
        seqs = [s.replace('\n', '') for s in seqs]
        seqs = sorted(list(set(seqs)))

    results = {}
    for model_dir in model_dirs:
        print(model_dir)
        model = GPT(base_model=model_dir)
        results[os.path.basename(model_dir)] = test_revaluation(seqs, model)
    return results

In [ ]:
def get_data_from_file(file_name):
    with open('data/' + file_name, 'rb') as f:
        all_results = pickle.load(f)

    df = pd.DataFrame(all_results)
    df = df.dropna(axis='columns')

    for col in df.columns:
        df[col] = df[col].apply(get_mean)

    stats_to_plot = df.describe()
    
    return stats_to_plot

def get_step_num(string):
    # Regular expression to match 'checkpoint-' followed by one or more digits
    match = re.match(r'checkpoint-(\d+)', string)
    
    # If there's a match and it's at the start of the string
    if match and match.start() == 0:
        return int(match.group(1))
    else:
        return 0
    
def produce_plot(description_string='-'):
    files = ['results_reward.pkl', 'results_transition.pkl', 'results_both.pkl']
    labels = ['Reward revaluation', 'Transition revaluation', 'Both revaluation']
    
    plt.figure(figsize=(4, 3))
    
    for ind, file in enumerate(files):
        stats_to_plot = get_data_from_file(file) 
        cols = stats_to_plot.columns.tolist()
        
        xs = [get_step_num(c) for c in cols]
        ys = [stats_to_plot[c]['mean'] for c in cols]
        yerrs = [stats_to_plot[c]['std'] / math.sqrt(stats_to_plot[c]['count']) for c in cols] 
        sorted_pairs = sorted(zip(xs, ys, yerrs), key=lambda x: x[0])  # Sort by xs
        sorted_xs, sorted_ys, sorted_yerrs = zip(*sorted_pairs)
        
        # Use errorbar instead of plot to include error margins
        plt.errorbar(sorted_xs[:-1], 
                     sorted_ys[:-1], 
                     yerr=sorted_yerrs[:-1],  # Specify the error bars
                     marker='o', 
                     capsize=5,
                     label=labels[ind])
    
    plt.yticks(np.linspace(0, 1, num=11))
    plt.ylabel('Accept / reject accuracy')
    plt.xlabel('Training steps')
    plt.legend()
    plt.savefig(f'revaluation over time{description_string}.png', dpi=500, bbox_inches='tight')
    plt.show()


In [ ]:
num_new = 1000
num_epochs = 20

for i in range(5):
    simulate_task(seed=i, 
                  num_new=num_new, 
                  num_epochs=num_epochs)       

In [ ]:
all_results_transition = []
all_results_reward = []
all_results_both = []

for i in range(5):
    base_dir = f'clm_script_{i}'
    
    results = test_consolidation(base_dir, task_type='transition')
    all_results_transition.append(results)

    results = test_consolidation(base_dir, task_type='reward')
    all_results_reward.append(results)

    results = test_consolidation(base_dir, task_type='both')
    all_results_both.append(results)

with open('data/results_transition.pkl', 'wb') as f:
    pickle.dump(all_results_transition, f)
with open('data/results_reward.pkl', 'wb') as f:
    pickle.dump(all_results_reward, f)
with open('data/results_both.pkl', 'wb') as f:
    pickle.dump(all_results_both, f)

In [ ]:
produce_plot(description_string='')

#### Regression coefficients approach

In [ ]:
def get_pred_reward_for_strategy(seq, output_dir, strategy):
    with open(os.path.join(output_dir, 'trial_info.pkl'), 'rb') as handle:
        trial_info = pickle.load(handle)
        stimuli = trial_info['stimuli']
        train_stop = trial_info['train_stop']
        train_reward = trial_info['train_reward']
    
    test_start = seq[0:seq.index(', STOP')].replace('START: ', '')
    test_stop = seq[seq.index('STOP'):seq.index('REWARD')].replace('STOP: ', '').replace(', ', '')
    test_reward = seq[seq.index('REWARD'):seq.index('SEQUENCE')].replace('REWARD: ', '').replace(', ', '')
    
    print(f'Train stop: {train_stop}, train reward: {train_reward}, test stop: {test_stop}, test reward: {test_reward}')
    print(stimuli)
    
    if strategy == 'revaluate_reward':
        # get train stop, test reward
        outcome = get_accept_reject(stimuli, test_start, train_stop, test_reward)
    if strategy == 'revaluate_transition':
        # get train reward, test stop
        outcome = get_accept_reject(stimuli, test_start, test_stop, train_reward)
    if strategy == 'revaluate_both':
        # get test reward, test stop
        outcome = get_accept_reject(stimuli, test_start, test_stop, test_reward)
    if strategy == 'no_revaluation':
        # get train reward, train stop
        outcome = get_accept_reject(stimuli, test_start, train_stop, train_reward)
    return outcome
        

In [ ]:
def get_accept_reject(stimuli, start, stop, reward):

    points = []
    sequence_started = False
    
    index = 0  # start index
    while True:
        stim = stimuli[index % len(stimuli)]
        colour, obj = stim.split()

        if not sequence_started:
            if stim == start:
                sequence_started = True
            index += 1
            continue
        else:
            if obj == reward:
                points.append(2)
            else:
                points.append(-1)
    
            if colour == stop:
                break

        index += 1  # move to the next stimulus

    print(f"Inferred points sequence for {start} / {stop} / {reward}: {points}")
    if sum(points) > 0:
        return 1
    if sum(points) < 0:
        return 0
    if sum(points) ==0:
        return 0.5

In [ ]:
def get_strategy(i, task_type='reward', strategy='no_revaluation'):
    model_dir = f'./clm_script_{i}'
    
    with open(os.path.join(model_dir, f'test_{task_type}.txt')) as f:
        seqs = f.readlines()
        print(f"{len(seqs)} seqs.")
        seqs = [s.replace('\n', '') for s in seqs]
        seqs = sorted(list(set(seqs)))
        print(task_type, strategy)
    
    preds = []
    for seq in seqs:
        pred = get_pred_reward_for_strategy(seq, model_dir, strategy)
        preds.append(pred)
        print(seq)
        print(f"Prediction: {pred}")
        print("......................")
    return np.asarray(preds)


def get_strategy_df(task_type='transition'):
    file_name = f'data/results_{task_type}.pkl'
    with open(file_name, 'rb') as f:
        all_results = pickle.load(f)
    df = pd.DataFrame(all_results)
    df = df.dropna(axis='columns')

    for col in df.columns:
        df[col] = df[col].apply(lambda x: np.array(x[1]))

    df["model_num"] = df.index
    print(df.index.tolist())

    df['NR_preds'] = df["model_num"].apply(lambda x: get_strategy(x, 
                                                                  task_type=task_type, 
                                                                  strategy='no_revaluation'))

    df['RR_preds'] = df["model_num"].apply(lambda x: get_strategy(x, 
                                                                  task_type=task_type, 
                                                                  strategy='revaluate_reward'))

    df['TR_preds'] = df["model_num"].apply(lambda x: get_strategy(x, 
                                                                  task_type=task_type, 
                                                                  strategy='revaluate_transition'))

    df['BR_preds'] = df["model_num"].apply(lambda x: get_strategy(x, 
                                                                  task_type=task_type, 
                                                                  strategy='revaluate_both'))
    return df

In [ ]:
df2 = get_strategy_df(task_type='transition')
df1 = get_strategy_df(task_type='reward')
df3 = get_strategy_df(task_type='both')
df = pd.concat([df1, df2, df3])

In [ ]:
def flatten_col(col):
    col_list = df[col].tolist()
    return [item for sublist in col_list for item in sublist]

df = df.reset_index()

for model in ['rule_model', 'checkpoint-100', 'checkpoint-200', 'checkpoint-300', 'checkpoint-400']:

    targets = flatten_col(model)
    feats = [flatten_col('RR_preds'),
             flatten_col('TR_preds'),
             flatten_col('NR_preds'),
             flatten_col('BR_preds')]
    feats = [list(row) for row in zip(*feats)]

    reg = LinearRegression(fit_intercept=True)
    reg.fit(feats, targets)
    print(model)
    print(reg.coef_)
    print(reg.intercept_)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso

def flatten_col(col):
    col_list = df[col].tolist()
    return [item for sublist in col_list for item in sublist]

# Function to plot the coefficients
def plot_coefficients(models, feature_names):
    coefs = []
    intercepts = []
    
    for model in models:
        targets = flatten_col(model)
        feats = [flatten_col('RR_preds'),
                 flatten_col('TR_preds'),
                 flatten_col('NR_preds'),
                 flatten_col('BR_preds')]
        feats = [list(row) for row in zip(*feats)]

        reg = LinearRegression(fit_intercept=True)
        reg.fit(feats, targets)
        
        coefs.append(reg.coef_)
        intercepts.append(reg.intercept_)

    # Plotting
    fig, axes = plt.subplots(nrows=1, ncols=len(models), figsize=(17, 3), sharey=True)
    for idx, ax in enumerate(axes):
        ax.bar(feature_names, coefs[idx])
        ax.axhline(y=intercepts[idx], color='r', linestyle='--')
        ax.set_title(models[idx])
        ax.set_xlabel('Features')
        ax.set_ylabel('Coefficient Value')
    plt.tight_layout()
    plt.show()

# Feature names
feature_names = ['RR', 'TR', 'MF', 'MB']
# Models
models = ['rule_model', 'checkpoint-100', 'checkpoint-200', 'checkpoint-300', 'checkpoint-400', 'checkpoint-500']

# Plot the coefficients
plot_coefficients(models, feature_names)


In [ ]:
models = ['rule_model', 'checkpoint-100', 'checkpoint-200', 'checkpoint-300', 'checkpoint-400']
strategies = ['NR_preds', 'RR_preds', 'TR_preds', 'BR_preds']

for model in models:
    for strategy in strategies:
        print(model, strategy)
        print(pearsonr(flatten_col(model), flatten_col(strategy)))

In [ ]:
def flatten_col(col):
    col_list = df[col].tolist()
    return [item for sublist in col_list for item in sublist]

# Define the new labels for models and strategies
model_labels = ['0', '100', '200', '300', '400']
strategy_labels = ['Model\nfree', 'Reward\nreval.', 'Transition\nreval.', 'Model\nbased']

# Original model and strategy identifiers
models = ['rule_model', 'checkpoint-100', 'checkpoint-200', 'checkpoint-300', 'checkpoint-400']
strategies = ['NR_preds', 'RR_preds', 'TR_preds', 'BR_preds']

# Prepare a DataFrame to store Pearson correlation coefficients with new labels
correlation_matrix = pd.DataFrame(index=model_labels, columns=strategy_labels)

# Calculate Pearson correlation coefficients
for model, model_label in zip(models, model_labels):
    for strategy, strategy_label in zip(strategies, strategy_labels):
        correlation, _ = pearsonr(flatten_col(model), flatten_col(strategy))
        correlation_matrix.loc[model_label, strategy_label] = correlation

# Convert the coefficients to float for plotting
correlation_matrix = correlation_matrix.astype(float)

# Create the heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(correlation_matrix.T, annot=True, cmap='coolwarm', center=0, fmt=".2f", annot_kws={"size": 10})
plt.xlabel('Training steps', fontsize=14)
plt.ylabel('Strategies', fontsize=14)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(rotation=0, fontsize=12)
plt.savefig('Correlation coefficients heatmap.png', dpi=500, bbox_inches='tight')
plt.show()


In [ ]:
def flatten_col(col):
    col_list = df[col].tolist()
    return [item for sublist in col_list for item in sublist]

# Prepare data for heatmap
model_steps = ['rule_model', 'checkpoint-100', 'checkpoint-200', 'checkpoint-300', 'checkpoint-400']
model_labels = ['0', '100', '200', '300', '400']
strategy_labels = ['NR_preds', 'RR_preds', 'TR_preds', 'BR_preds']
strategy_names = ['Model\nfree', 'Reward\nreval.', 'Transition\nreval.', 'Model\nbased']

coefficients = pd.DataFrame(columns=model_labels, index=strategy_names)

for model, label in zip(model_steps, model_labels):
    targets = flatten_col(model)
    feats = [flatten_col(strategy) for strategy in strategy_labels]
    feats = [list(row) for row in zip(*feats)]

    reg = LinearRegression(fit_intercept=True)
    reg.fit(feats, targets)
    coefficients[label] = reg.coef_

# Plotting the heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(coefficients, annot=True, cmap='coolwarm', center=0, fmt=".2f", annot_kws={"size": 10})
plt.xlabel('Training steps', fontsize=14)
plt.ylabel('Strategies', fontsize=14)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(rotation=0, fontsize=12)
plt.savefig('Regression coefficients heatmap.png', dpi=500, bbox_inches='tight')
plt.show()


#### 'Retrieval augmented generation'

Can the generative network's outputs be conditioned on hippocampal sequences (inspired by retrieval augmented generation), to support some degree of generalisation soon after encoding?

In [ ]:
model = GPT(base_model='rule_model')

In [ ]:
def get_most_similar(train_seqs, test_seq, n):
    # Extract words from the test_seq up to "SEQUENCE"
    test_seq_words = set(test_seq.split("SEQUENCE:")[0].split())
    start_condition = test_seq.split("STOP:")[0]
    train_seqs = [t for t in train_seqs if start_condition in t]

    overlap_scores = []

    for train_seq in train_seqs:
        # Extract words from the train_seq up to "SEQUENCE"
        train_seq_words = set(train_seq.split("SEQUENCE:")[0].split())
        
        # Calculate the overlap by finding the intersection of sets
        overlap = len(test_seq_words.intersection(train_seq_words))
        
        # Store the train sequence with its overlap score
        overlap_scores.append((train_seq, overlap))

    # Sort the list of tuples by overlap score in descending order and select the top n
    top_n_seqs = sorted(overlap_scores, key=lambda x: x[1], reverse=True)[:n]
    
    # Return only the sequences, not their scores
    return [seq[0] for seq in top_n_seqs]

# Example usage
train_seqs = [
    "START: blue animal, STOP: green, REWARD: vehicle, SEQUENCE: blue vehicle (3)",
    "START: green animal, STOP: red, REWARD: vehicle, SEQUENCE: red vehicle (2)",
    "START: yellow animal, STOP: red, REWARD: vegetable, SEQUENCE: red vegetable (2)",
    "START: green animal, STOP: blue, REWARD: vehicle, SEQUENCE: blue vehicle (1)",
    "START: green animal, STOP: red, REWARD: bicycle, SEQUENCE: red bicycle (2)"
]
test_seq = "START: green animal, STOP: red, REWARD: vehicle, SEQUENCE: red vehicle (2)"

print(get_most_similar(train_seqs, test_seq, n=9))

In [ ]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def get_seqs_RAG(base_dir='clm_script_0', test_type='reward'):
    with open(os.path.join(base_dir, f'test_{test_type}.txt')) as f:
        seqs = f.readlines()
        seqs = [s.replace('\n', '') for s in seqs]
        test_seqs = sorted(list(set(seqs)))

    with open(os.path.join(base_dir, f'train.txt')) as f:
        seqs = f.readlines()
        seqs = [s.replace('\n', '') for s in seqs]
        train_seqs = sorted(list(set(seqs)))
    
    return train_seqs, test_seqs
        
def test_RAG(base_dir = 'clm_script_0', test_type='transition', n=1):
    train_seqs, test_seqs = get_seqs_RAG(base_dir=base_dir, test_type=test_type)
    print(len(train_seqs))

    result_bools = []
    mf_baseline_bools = []
    nc_baseline_bools = []

    for test in test_seqs:
        print(f"Test sequence: {test}")
        train_examples = get_most_similar(train_seqs, test, n=n)
        print(f"Train example: {train_examples}")
        
        continuation = model.continue_input("\n".join(train_examples) + '\n' + test[0:test.index('SEQUENCE')] + 'SEQUENCE:',
                                            do_sample=False)
        start_ind = find_nth(continuation, 'SEQUENCE', len(train_examples) + 1)
        end_ind = find_nth(continuation, 'START', len(train_examples) + 2)

        true_a_v_r = get_accept_reject_choice(test)
        pred_a_v_r = get_accept_reject_choice(continuation[start_ind:end_ind])
        if true_a_v_r == pred_a_v_r:
            result_bools.append(1)
        else:
            result_bools.append(0)

        mf_baseline_a_v_r = get_accept_reject_choice(train_examples[0])
        if true_a_v_r == mf_baseline_a_v_r:
            mf_baseline_bools.append(1)
        else:
            mf_baseline_bools.append(0)
        
        continuation = model.continue_input(test[0:test.index('SEQUENCE')] + 'SEQUENCE:',
                                            do_sample=False)
        start_ind = find_nth(continuation, 'SEQUENCE', 1)
        end_ind = find_nth(continuation, 'START', 2)
        nc_baseline_a_v_r = get_accept_reject_choice(continuation[start_ind:end_ind])
        if true_a_v_r == nc_baseline_a_v_r:
            nc_baseline_bools.append(1)
        else:
            nc_baseline_bools.append(0)
            
    return result_bools, mf_baseline_bools, nc_baseline_bools

all_transition = []
all_reward = []
all_both = []

for i in range(5):
    res = test_RAG(base_dir = f'clm_script_{i}', test_type='reward', n=1)
    all_reward.append(res)
    res = test_RAG(base_dir = f'clm_script_{i}', test_type='transition', n=1)
    all_transition.append(res)
    res = test_RAG(base_dir = f'clm_script_{i}', test_type='both', n=1)
    all_both.append(res)

In [ ]:
with open('data/rag_reward.pkl', 'wb') as f:
    pickle.dump(all_reward, f)

with open('data/rag_transition.pkl', 'wb') as f:
    pickle.dump(all_transition, f)

with open('data/rag_both.pkl', 'wb') as f:
    pickle.dump(all_both, f)

In [ ]:
with open('data/rag_reward.pkl', 'rb') as f:
    all_reward = pickle.load(f)

with open('data/rag_transition.pkl', 'rb') as f:
    all_transition = pickle.load(f)

with open('data/rag_both.pkl', 'rb') as f:
    all_both = pickle.load(f)

In [ ]:
# Calculate accuracies
transition_accuracies_rag = [np.mean(t[0]) for t in all_transition]
transition_accuracies_mf = [np.mean(t[1]) for t in all_transition]
transition_accuracies_nc = [np.mean(t[2]) for t in all_transition]

reward_accuracies_rag = [np.mean(r[0]) for r in all_reward]
reward_accuracies_mf = [np.mean(r[1]) for r in all_reward]
reward_accuracies_nc = [np.mean(r[2]) for r in all_reward]

both_accuracies_rag = [np.mean(r[0]) for r in all_both]
both_accuracies_mf = [np.mean(r[1]) for r in all_both]
both_accuracies_nc = [np.mean(r[2]) for r in all_both]

# Calculate means
transition_means_rag = np.mean(transition_accuracies_rag)
transition_means_mf = np.mean(transition_accuracies_mf)
transition_means_nc = np.mean(transition_accuracies_nc)

reward_means_rag = np.mean(reward_accuracies_rag)
reward_means_mf = np.mean(reward_accuracies_mf)
reward_means_nc = np.mean(reward_accuracies_nc)

both_means_rag = np.mean(both_accuracies_rag)
both_means_mf = np.mean(both_accuracies_mf)
both_means_nc = np.mean(both_accuracies_nc)

# Calculate SEMs
transition_sem_rag = np.std(transition_accuracies_rag) / np.sqrt(len(transition_accuracies_rag))
transition_sem_mf = np.std(transition_accuracies_mf) / np.sqrt(len(transition_accuracies_mf))
transition_sem_nc = np.std(transition_accuracies_nc) / np.sqrt(len(transition_accuracies_nc))

reward_sem_rag = np.std(reward_accuracies_rag) / np.sqrt(len(reward_accuracies_rag))
reward_sem_mf = np.std(reward_accuracies_mf) / np.sqrt(len(reward_accuracies_mf))
reward_sem_nc = np.std(reward_accuracies_nc) / np.sqrt(len(reward_accuracies_nc))

both_sem_rag = np.std(both_accuracies_rag) / np.sqrt(len(both_accuracies_rag))
both_sem_mf = np.std(both_accuracies_mf) / np.sqrt(len(both_accuracies_mf))
both_sem_nc = np.std(both_accuracies_nc) / np.sqrt(len(both_accuracies_nc))

# Data to plot
labels = ['NC only', 'HPC only', 'RAG']

transition_means = [transition_means_nc, transition_means_mf, transition_means_rag]
reward_means = [reward_means_nc, reward_means_mf, reward_means_rag]
both_means = [both_means_nc, both_means_mf, both_means_rag]

transition_sems = [transition_sem_nc, transition_sem_mf, transition_sem_rag]
reward_sems = [reward_sem_nc, reward_sem_mf, reward_sem_rag]
both_sems = [both_sem_nc, both_sem_mf, both_sem_rag]

# Set up the plot
fig, ax = plt.subplots(figsize=(6, 3))
bar_width = 0.25  # Adjusted bar width for better visual separation
index = np.arange(len(labels))
opacity = 0.4

# Plot bars with SEM error bars
bars1 = ax.bar(index - bar_width, transition_means, bar_width, alpha=opacity, yerr=transition_sems, capsize=2, 
               label='Transition', color='red')
bars2 = ax.bar(index, reward_means, bar_width, alpha=opacity, yerr=reward_sems, capsize=2, label='Reward',
              color='blue')
bars3 = ax.bar(index + bar_width, both_means, bar_width, alpha=opacity, yerr=both_sems, capsize=2, label='Both',
              color='purple')

# Finalizing the plot
ax.set_xlabel('Method')
ax.set_ylabel('Accuracy')
ax.set_xticks(index)
ax.set_xticklabels(labels)
ax.set_ylim(0, 1.0)
ax.legend(title="Revaluation Type")

plt.tight_layout()
plt.savefig('RAG_graph_by_method_plan.png', dpi=300)
plt.show()
